Imports

In [49]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision 
from torch.utils.data import DataLoader
from tqdm import tqdm

Create fully Connected Network

In [86]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 200)
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, num_classes)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        x=F.relu(self.fc1(x))
        x = self.dropout(x)
        x=F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

Set device

In [87]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Hyperparameters

In [88]:
input_size=784
num_classes=10
lr=0.001
batch_size=64
num_epochs=10

Load Data

In [89]:
train_dataset= datasets.MNIST(root='../datasets/', 
                             train= True,
                             download=True,
                             transform=transforms.ToTensor())
test_dataset= datasets.MNIST(root='../datasets/', 
                             train= False,
                             download=True,
                             transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
test_loader  = DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle= True)


Initialize the network

In [90]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

Loss and optimizer

In [91]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

Train Network


In [92]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.to(device=device)
        targets = targets.to(device=device)
        data = data.reshape(data.shape[0], -1) #flatten 
        
        #forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        

100%|███████████████████████████████████████████████████████████████████████████████| 938/938 [00:05<00:00, 160.44it/s]


Check aacuracy on training & test to see how good our model

In [94]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    
    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

100%|███████████████████████████████████████████████████████████████████████████████| 938/938 [00:04<00:00, 202.16it/s]


Accuracy on training set: 99.34


100%|███████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 199.65it/s]

Accuracy on test set: 98.08
